# Resilient Tube

In [1]:
from csv import reader
from igraph import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from random import choice

In [2]:
# Read local file and create edge list in tuple format 
df = pd.read_csv("Data/InputData_Edges.csv")
dfedge = df[['From (RODS name)', 'To (RODS name)', 'Name']]
tuples = [tuple(x) for x in dfedge.values]

In [3]:
# Create graph based on edge list from previous step
ptn = Graph.TupleList(tuples, directed=True, vertex_name_attr="name", edge_attrs="line")
ptn["name"] = "PTN London Underground"
ptn.summary()

'IGRAPH DN-- 268 731 -- PTN London Underground\n+ attr: name (g), name (v), line (e)'

In [4]:
# Add flooding risk probability to nodes
df2 = pd.read_csv("Data/InputData_Stations.csv")
floods = np.asarray(df2[['Name (RODS)','FloodRisk1000a']])
for station in floods:
    try:
        vs = ptn.vs.find(name=station[0])
        vs['FloodRisk1000a'] = station[1]
    except ValueError:
        print(station[0], ' - not found!')

list(ptn.vs)

[igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 0, {'name': 'Harrow & Wealdstone', 'FloodRisk1000a': 0.1364}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 1, {'name': 'Kenton', 'FloodRisk1000a': 0.1935}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 2, {'name': 'South Kenton', 'FloodRisk1000a': 0.4833}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 3, {'name': 'North Wembley', 'FloodRisk1000a': 0.2384}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 4, {'name': 'Wembley Central', 'FloodRisk1000a': 0.0472}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 5, {'name': 'Stonebridge Park', 'FloodRisk1000a': 0.2317}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 6, {'name': 'Harlesden', 'FloodRisk1000a': 0.124}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 7, {'name': 'Willesden Junction', 'FloodRisk1000a': 0.205}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 8, {'name': 'Kensal Green', 'FloodRisk1000a': 0.0771}),
 igraph.V

In [9]:
# Add betweeness centrality to vertices
# for vertex in ptn.vs:
#     vertex["betw_cent"] = ptn.betweenness(vertex, directed=True)

# Add coordinates to stations
coords = np.asarray(df2[['Name (RODS)','Lat','Long']])
for station in coords:
#     print(station[1:3])
    try:
        vs = ptn.vs.find(name=station[0])
        vs['coords'] = list(station[1:3])
    except ValueError:
        print(station[0], ' - not found!')

list(ptn.vs)

[igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 0, {'coords': [51.5925, -0.3351], 'name': 'Harrow & Wealdstone', 'FloodRisk1000a': 0.1364}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 1, {'coords': [51.5816, -0.3162], 'name': 'Kenton', 'FloodRisk1000a': 0.1935}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 2, {'coords': [51.5701, -0.3081], 'name': 'South Kenton', 'FloodRisk1000a': 0.4833}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 3, {'coords': [51.5621, -0.3034], 'name': 'North Wembley', 'FloodRisk1000a': 0.2384}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 4, {'coords': [51.5519, -0.2963], 'name': 'Wembley Central', 'FloodRisk1000a': 0.0472}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 5, {'coords': [51.5439, -0.2759], 'name': 'Stonebridge Park', 'FloodRisk1000a': 0.2317}),
 igraph.Vertex(<igraph.Graph object at 0x108b4d318>, 6, {'coords': [51.5362, -0.2575], 'name': 'Harlesden', 'FloodRisk1000a': 0.124}),
 igraph.Vertex(<igraph

**Create target list by floooding**

In [ ]:
# Process used during dacas meeting, will be replaced by vertex probaility
df3 = pd.read_csv("Data/InputData_Stations.csv")
flood = df3[['name','FloodRisk1000a']]
threshold = flood['name'][flood.FloodRisk1000a > 0.3]
targets = []

for i in threshold:
    targets.append(i)

ptn2 = ptn
targets

In [ ]:
# Proting function

def plot_hist(tup, target):
    sns.set(context="notebook", font_scale=1.2)
    plt.figure(facecolor="white", figsize=(9, 6), dpi=300)
    plt.xlabel('Path Length')
    plt.ylabel('Frequency')
    plt.title('Path Length distribution station: %s' % str(target))
    plt.bar(*zip(*tup))

**Delete nodes based on Flood Risk**

In [ ]:
% matplotlib inline

for i in targets:
    path_hist_orig = list(ptn.path_length_hist(directed=True).bins())
    hist_orig = []
    for bin1 in path_hist_orig:
        x1 = bin1[0]
        y1 = bin1[2]
        hist_orig.append((x1,y1))
    plot_hist(hist_orig, i + str(' original'))

    ptn.delete_vertices(ptn.vs.find(name=i))
    path_hist_dell = list(ptn.path_length_hist(directed=True).bins())
    hist_dell = []
    for bin2 in path_hist_dell:
        x2 = bin2[0]
        y2 = bin2[2]
        hist_dell.append((x2,y2))
    
    plot_hist(hist_dell, 'Damaged')

**Delete nodes based on Betweeness Centrality**

In [ ]:
% matplotlib inline

count = 0

while count <= 10:
    max_bc = max(ptn2.vs['beetw_cent'])
    target = ptn2.vs(ptn2.vs['beetw_cent'] == max_bc)['name']
    
    path_hist_orig = list(ptn2.path_length_hist(directed=True).bins())
    hist_orig = []
    for bin1 in path_hist_orig:
        x1 = bin1[0]
        y1 = bin1[2]
        hist_orig.append((x1,y1))
    plot_hist(hist_orig, i + str(' original'))

    ptn2.delete_vertices(ptn2.vs.find(name=i))
    path_hist_dell = list(ptn2.path_length_hist(directed=True).bins())
    hist_dell = []
    for bin2 in path_hist_dell:
        x2 = bin2[0]
        y2 = bin2[2]
        hist_dell.append((x2,y2))
    plot_hist(hist_dell, 'Damaged')
    
    count += 1

In [ ]:
# Ploting function
# % matplotlibe inline
# layout = ptn.layout_kamada_kawai()
# plot(ptn, layout = layout)